In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

class UserOverviewAnalysis:
    def __init__(self, mydata):
        self.mydata = mydata

    def clean_and_preprocess(self):
        # Placeholder for data cleaning and preprocessing logic
        # Implement your logic to clean and preprocess the data
        cleaned_data = self.mydata  # Replace this with your actual cleaning/preprocessing logic
        return cleaned_data

    def visualize_results(self):
        # Common visualization logic that can be reused
        sns.set_theme(style="whitegrid")
        plt.figure(figsize=(10, 6))
        # Placeholder plotting logic
        # Implement your visualization logic
        plt.show()

    def aggregate_user_behaviour(self):
        # Placeholder for user behaviour aggregation
        # Implement your logic to aggregate user behavior
        aggregated_data = self.mydata.groupby('user_id').sum()  # Replace this with your actual aggregation logic
        return aggregated_data

    def top_10_handsets(self):
        # Placeholder for identifying top 10 handsets
        # Implement your logic to identify the top 10 handsets
        top_10_handsets = self.mydata['handset_model'].value_counts().head(10)  # Replace this with your actual logic
        return top_10_handsets

    def top_3_manufacturers(self):
        # Placeholder for identifying top 3 handset manufacturers
        # Implement your logic to identify the top 3 manufacturers
        top_3_manufacturers = self.mydata['manufacturer'].value_counts().head(3)  # Replace this with your actual logic
        return top_3_manufacturers

    def top_5_handsets_per_manufacturer(self):
        # Placeholder for identifying top 5 handsets per top 3 handset manufacturer
        # Implement your logic to identify the top 5 handsets per top 3 manufacturers
        top_manufacturers = self.top_3_manufacturers().index
        top_5_handsets_per_manufacturer = (
            self.mydata[self.mydata['manufacturer'].isin(top_manufacturers)]
            .groupby(['manufacturer', 'handset_model'])
            .size()
            .groupby('manufacturer', group_keys=False)
            .nlargest(5)
        )  # Replace this with your actual logic
        return top_5_handsets_per_manufacturer

    def data_analysis(self):
        # Example: Analyze and visualize the distribution of session durations for each application
        applications = ['SocialMediaData', 'GoogleData', 'EmailData', 'YoutubeData', 'NetflixData', 'GamingData', 'OtherData']

        plt.figure(figsize=(12, 8))
        for app in applications:
            sns.histplot(self.mydata[self.mydata[app] > 0]['SessionDuration'], label=app, kde=True)

        plt.title('Distribution of Session Durations for Each Application')
        plt.xlabel('Session Duration')
        plt.ylabel('Frequency')
        plt.legend()
        plt.show()

# Database connection parameters
db_params = {
    'dbname': 'week1',
    'user': 'postgres',
    'password': 'habte',
    'host': 'localhost',
    'port': '5432'
}

# Create a SQLAlchemy engine
engine = create_engine(f'postgresql+psycopg2://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}:{db_params["port"]}/{db_params["dbname"]}')

# SQL query to retrieve data
sql_query = "SELECT * FROM xdr_data;"

# Read data from PostgreSQL into a DataFrame
mydata = pd.read_sql_query(sql_query, engine)

# Create an instance of UserOverviewAnalysis with the actual data
user_analysis = UserOverviewAnalysis(mydata)

# Execute the analysis
cleaned_data = user_analysis.clean_and_preprocess()
aggregated_data = user_analysis.aggregate_user_behaviour()
top_10_handsets = user_analysis.top_10_handsets()
top_3_manufacturers = user_analysis.top_3_manufacturers()
top_5_handsets_per_manufacturer = user_analysis.top_5_handsets_per_manufacturer()
user_analysis.visualize_results()
user_analysis.data_analysis()


KeyError: 'user_id'